# Proyecto 2
### Ingeniería de Datos
> *Estudiante: Víctor Manuel Rodríguez Umaña*

> *Carnet: 24007670*

> *Curso: Ciencia de Datos en Python*

In [7]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2

### Lectura de archivo de configuración, conexión y validación de instancias de la BD

In [20]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [10]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                           aws_secret_access_key=config.get('IAM', 'SECRET_KEY'),
                           region_name='us-east-1')

In [11]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()
for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])
print(f"Instancias disponibles: {rds_instances_ids}")

Instancias disponibles: ['tienda-db-pg']


### Creación de la instancia de la BD

In [24]:
try:
    response = aws_rds_conn.create_db_instance(DBInstanceIdentifier=config.get('TIENDA', 'DB_INSTANCE_ID'),
                                               DBName=config.get('TIENDA', 'DB_NAME'),
                                               MasterUsername=config.get('TIENDA', 'DB_USERNAME'),
                                               MasterUserPassword=config.get('TIENDA', 'DB_PASSWORD'),
                                               Port=int(config.get('TIENDA', 'DB_PORT')),
                                               DBInstanceClass='db.t3.micro',
                                               Engine='postgres',
                                               PubliclyAccessible=True,
                                               AllocatedStorage=20,
                                               VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                                            )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print("La instancia ya existe")
except Exception as ex:
    print("Error:", ex)

{'DBInstance': {'DBInstanceIdentifier': 'tienda-db-pg', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'postgres_admin', 'DBName': 'tienda', 'AllocatedStorage': 20, 'PreferredBackupWindow': '07:11-07:41', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-07ad5a13995b326d6', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0b6b8a53854b596e9', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-018a924b61683ebfd', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-02b843d027c14408a', 'SubnetAvailabilityZone': {'Name': 'us-east-1a'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'sub